<a href="https://colab.research.google.com/github/ThomasAbeyta/Suitable_Jaguar_Habitat/blob/main/Predictive_Model_for_Suitable_Jaguar_Habitat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Image Classification with Deep Learning

## Mission: 
The primary objective of this study is to undersand what is a Jaguar habitat along the Guatamalen border. Not having a background in ecology nor much knowledge on juguars I will begin by building a comprehension of the ecological niche, behavioral patterns, and environmental relationships of the jaguar. Toward this end, I will begin building a predictive model that can be established to aid conservation initiatives.

## Background:
This project is a collaboration from a past client and long time friend Jesse Sprague. He is an Earth and Planetary Science graduate with a strong interest in the intersection of technology and nature. He has extensive experience in remote sensing data and has worked as a spatial data scientist, software engineer, and architect. Currently, as the founder of ASIO, he is dedicated to promoting collaboration and spatial data integration to ensure environmental stewardship and sustainable living.

The GPS data comes from Dr. Gerardo Ceballos he is a renowned ecologist and Full-Time "C" Senior Researcher at the UNAM Institute of Ecology. 


Similiar research:
https://link.springer.com/article/10.1007/s10980-021-01335-2#data-availability

Stat analysis (why not to use centroid)
- Activity patterns: Jaguars are elusive and primarily nocturnal predators, and their activity patterns can vary depending on prey availability, breeding season, or other environmental factors. 
- Temporal resolution:if the jaguar is more active at night and rests during the day, the centroid might be biased towards its resting locations.
- Individual variation: The behavior of individual jaguars might differ due to factors such as sex, age, or territorial status.



## Neural Networks
- Nonlinearity: Neural networks can capture complex, non-linear relationships between environmental variables and species presence or habitat suitability, which may be difficult to represent using traditional statistical methods.

##Data Dictionary 
 


### Definitions:


# The Data

## Initial Data Exploration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Mount libs

In [ ]:
!pip install rasterio geopandas shapely
!pip install pandas geopandas pysal shapely
!pip install pandas geopandas pysal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 33.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 16.2 MB/s eta 0:00:00

#Imports

In [ ]:
import pickle
import rasterio
import geopandas as gpd
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as stats
import statsmodels.api as sm
import plotly.graph_objects as go
import tensorflow as tf
import tensorflow.keras as keras
import pickle
import os
import re

from rasterio.mask import mask


import pysal as ps
import pysal.lib as lib

from pysal.explore import esda
import libpysal

from shapely import wkt
from shapely.geometry import Point


from rasterio.windows import Window
from rasterio.transform import xy
from shapely.geometry import Point, MultiPolygon, Polygon


from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import plot_model, load_img, img_to_array
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn import datasets, metrics, model_selection
from sklearn import model_selection

/usr/local/lib/python3.10/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-3-2b4188f55c90>:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling Py

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

#Check GPU

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


#Imaging Data

Preparing the image for analysis
- Import 5 bands
- Clip images to research area
- reproject the coordinates: The ESPG area reprojects the the coordinate system to metrics (EPSG:32616 - WGS 84 / UTM zone 16N, Accurate to 2meters)
- Merge while selecting band seperation

Plot points
- Import coordinates (lat,long)
- Reproject points: EPSG:32616 - WGS 84 / UTM zone 16N
- Create a buffer around each point 50x50 meters wide
- Create a .geojson file of points

Export to folder:
- Export mergered image as .tiff
- Export geojson of points


#Pickle data

In [ ]:
# Load the DataFrame
with open('/content/drive/MyDrive/Data_Science_Bootcamp/Projects/Capstone_Jaguar/Jaguar_observed_df.pkl', 'rb') as f:
    observed_df = pickle.load(f)


In [ ]:
# Load the DataFrame from the .pkl file
with open('/content/drive/MyDrive/Data_Science_Bootcamp/Projects/Capstone_Jaguar/jaguar_false_observations_db.pkl', 'rb') as f:
    unseen_df = pickle.load(f)


#Merge and Clean DF

In [ ]:
observed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773 entries, 0 to 1772
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   chip_data            1773 non-null   object  
 1   ndvi_value           1773 non-null   float64 
 2   wkt_geom             1773 non-null   object  
 3   date                 1773 non-null   object  
 4   x                    1773 non-null   float64 
 5   y                    1773 non-null   float64 
 6   geometry             1773 non-null   geometry
 7   timestamp            1773 non-null   float64 
 8   day                  1773 non-null   int64   
 9   week                 1773 non-null   UInt32  
 10  dist_between_points  1773 non-null   float64 
 11  topo_DEM             1773 non-null   float64 
 12  observed             1773 non-null   int64   
dtypes: UInt32(1), float64(6), geometry(1), int64(2), object(3)
memory usage: 175.0+ KB


In [ ]:
observed_df.shape

(1773, 13)

In [ ]:
unseen_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   chip_data            1700 non-null   object        
 1   ndvi_value           1700 non-null   float64       
 2   topo_DEM             1700 non-null   float64       
 3   wkt_geom             1700 non-null   object        
 4   date                 1700 non-null   datetime64[ns]
 5   x                    1700 non-null   float64       
 6   y                    1700 non-null   float64       
 7   geometry             1700 non-null   geometry      
 8   timestamp            1700 non-null   float64       
 9   day                  1700 non-null   int64         
 10  week                 1700 non-null   UInt32        
 11  dist_between_points  1700 non-null   float64       
 12  chip_data            1700 non-null   object        
 13  observed             1700 non-nul

In [ ]:
unseen_df.shape

(1700, 14)

In [ ]:
unseen_df_clean = unseen_df.loc[:, ~unseen_df.columns.duplicated()]


In [ ]:
unseen_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   chip_data            1700 non-null   object        
 1   ndvi_value           1700 non-null   float64       
 2   topo_DEM             1700 non-null   float64       
 3   wkt_geom             1700 non-null   object        
 4   date                 1700 non-null   datetime64[ns]
 5   x                    1700 non-null   float64       
 6   y                    1700 non-null   float64       
 7   geometry             1700 non-null   geometry      
 8   timestamp            1700 non-null   float64       
 9   day                  1700 non-null   int64         
 10  week                 1700 non-null   UInt32        
 11  dist_between_points  1700 non-null   float64       
 12  observed             1700 non-null   int64         
dtypes: UInt32(1), datetime64[ns](1), 

In [ ]:
# reorder the columns 
# df = fo_df.reindex(columns=jag_df.columns)
df1 = unseen_df_clean.reindex(columns=['date','chip_data','ndvi_value','wkt_geom','x','y','geometry','day','week','dist_between_points','topo_DEM','observed'])
df1.head(1)

,date,chip_data,ndvi_value,wkt_geom,x,y,geometry,day,week,dist_between_points,topo_DEM,observed
0,2013-05-11,"[[[6080, 4573, 3827, 4151, 4829], [6160, 4694,...",0.244891,Point (284728.91250939114252105 2091830.923457...,284728.912509,2.091831e+06,POINT (284728.913 2091830.923),11,19,0.0,83.0,0


In [ ]:
df2 = observed_df.reindex(columns=df1.columns)
df2.head(1)

,date,chip_data,ndvi_value,wkt_geom,x,y,geometry,day,week,dist_between_points,topo_DEM,observed
0,2013-05-11,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...",0.370446,Point (275202.23302631557453424 2064878.131713...,275202.233026,2.064878e+06,POINT (275202.233 2064878.132),11,19,0.0,139.0,1


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773 entries, 0 to 1772
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   date                 1773 non-null   object  
 1   chip_data            1773 non-null   object  
 2   ndvi_value           1773 non-null   float64 
 3   wkt_geom             1773 non-null   object  
 4   x                    1773 non-null   float64 
 5   y                    1773 non-null   float64 
 6   geometry             1773 non-null   geometry
 7   day                  1773 non-null   int64   
 8   week                 1773 non-null   UInt32  
 9   dist_between_points  1773 non-null   float64 
 10  topo_DEM             1773 non-null   float64 
 11  observed             1773 non-null   int64   
dtypes: UInt32(1), float64(5), geometry(1), int64(2), object(3)
memory usage: 161.1+ KB


In [ ]:
jag_complete_df = pd.concat([df1, df2], axis=0)

In [ ]:
jag_complete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3473 entries, 0 to 1772
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   date                 3473 non-null   object  
 1   chip_data            3473 non-null   object  
 2   ndvi_value           3473 non-null   float64 
 3   wkt_geom             3473 non-null   object  
 4   x                    3473 non-null   float64 
 5   y                    3473 non-null   float64 
 6   geometry             3473 non-null   geometry
 7   day                  3473 non-null   int64   
 8   week                 3473 non-null   UInt32  
 9   dist_between_points  3473 non-null   float64 
 10  topo_DEM             3473 non-null   float64 
 11  observed             3473 non-null   int64   
dtypes: UInt32(1), float64(5), geometry(1), int64(2), object(3)
memory usage: 342.6+ KB


In [ ]:
jag_complete_df.tail(2)

,date,chip_data,ndvi_value,wkt_geom,x,y,geometry,day,week,dist_between_points,topo_DEM,observed
1771,2014-05-05,"[[[0, 0, 0, 0, 0], [5078, 4211, 2285, 4673, 10...",0.383713,Point (275191.15383531514089555 2064833.975330...,275191.153835,2.064834e+06,POINT (275191.154 2064833.975),5,19,23.832830,136.0,1
1772,2014-05-05,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...",0.365515,Point (275191.4175098433624953 2064856.1163516...,275191.417510,2.064856e+06,POINT (275191.418 2064856.116),5,19,45.525103,136.0,1


In [ ]:
jag_complete_df.shape

(3473, 12)

In [ ]:
# Convert 'date' column to Unix timestamps
jag_complete_df['timestamp'] = jag_complete_df['date'].apply(lambda x: pd.Timestamp(x).timestamp())

In [ ]:
clean_df1=jag_complete_df.drop(['date'], axis=1)

In [ ]:
clean_df=clean_df1.copy()

In [ ]:
clean_df.tail(1)

,chip_data,ndvi_value,wkt_geom,x,y,geometry,day,week,dist_between_points,topo_DEM,observed
1772,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...",0.365515,Point (275191.4175098433624953 2064856.1163516...,275191.41751,2.064856e+06,POINT (275191.418 2064856.116),5,19,45.525103,136.0,1


In [ ]:
clean_df1=clean_df.drop(['wkt_geom','geometry'], axis=1)

In [ ]:
clean_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3473 entries, 0 to 1772
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   chip_data            3473 non-null   object 
 1   ndvi_value           3473 non-null   float64
 2   x                    3473 non-null   float64
 3   y                    3473 non-null   float64
 4   day                  3473 non-null   int64  
 5   week                 3473 non-null   UInt32 
 6   dist_between_points  3473 non-null   float64
 7   topo_DEM             3473 non-null   float64
 8   observed             3473 non-null   int64  
dtypes: UInt32(1), float64(5), int64(2), object(1)
memory usage: 261.2+ KB


In [ ]:
#convert to string and replace characters
clean_df['geometry'] = clean_df['geometry'].astype(str).str.replace('POINT ', '')
clean_df['wkt_geom'] = clean_df['wkt_geom'].astype(str).str.replace('Point ', '')
clean_df['geometry'] = clean_df['geometry'].astype(str).str.replace('(', '')
clean_df['wkt_geom'] = clean_df['wkt_geom'].astype(str).str.replace('(', '')
clean_df['geometry'] = clean_df['geometry'].astype(str).str.replace(')', '')
clean_df['wkt_geom'] = clean_df['wkt_geom'].astype(str).str.replace(')', '')

<ipython-input-27-6002699f3993>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clean_df['geometry'] = clean_df['geometry'].astype(str).str.replace('(', '')
<ipython-input-27-6002699f3993>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clean_df['wkt_geom'] = clean_df['wkt_geom'].astype(str).str.replace('(', '')
<ipython-input-27-6002699f3993>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clean_df['geometry'] = clean_df['geometry'].astype(str).str.replace(')', '')
<ipython-input-27-6002699f3993>:7: FutureWarning: Th

In [ ]:
columns_to_convert= """chip_data,	ndvi_value, wkt_geom,	x,	y,	geometry,	day,	week,	dist_between_points,	topo_DEM,	observed,	timestamp""".split(',')  
columns_to_convert = [feature.strip() for feature in columns_to_convert]
columns_to_convert

['chip_data',
 'ndvi_value',
 'wkt_geom',
 'x',
 'y',
 'geometry',
 'day',
 'week',
 'dist_between_points',
 'topo_DEM',
 'observed',
 'timestamp']

In [ ]:
# Convert features to numeric data type
clean_df[columns_to_convert] = clean_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3473 entries, 0 to 1772
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   chip_data            0 non-null      float64
 1   ndvi_value           3473 non-null   float64
 2   wkt_geom             0 non-null      float64
 3   x                    3473 non-null   float64
 4   y                    3473 non-null   float64
 5   geometry             0 non-null      float64
 6   day                  3473 non-null   int64  
 7   week                 3473 non-null   UInt32 
 8   dist_between_points  3473 non-null   float64
 9   topo_DEM             1700 non-null   float64
 10  observed             3473 non-null   int64  
 11  timestamp            3473 non-null   float64
dtypes: UInt32(1), float64(9), int64(2)
memory usage: 342.6 KB


In [ ]:
clean_df.head(1)


,chip_data,ndvi_value,wkt_geom,x,y,geometry,day,week,dist_between_points,topo_DEM,observed,timestamp
0,NaN,0.244891,NaN,284728.912509,2.091831e+06,NaN,11,19,0.0,83.0,0,1.368230e+09


In [ ]:
# apply the load_npy function to the 'filename' column of the DataFrame
clean = clean_df1['chip_data'].apply(lambda x: np.load(io.BytesIO(x)))


NameError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Preprocessing the pixel chips (resize, normalize, etc.)
def preprocess_chip(chip_data, target_size):
    # Add your preprocessing logic here
    pass

# Preprocess all chip_data in the DataFrame
clean_df1["chip_data"] = clean_df1["chip_data"].apply(lambda x: preprocess_chip(x, target_size))

In [ ]:
#clean_df=clean_df1.copy()

In [ ]:
clean_df.head(1)

In [ ]:
clean_df.isna().sum()

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.expand_frame_repr", False)

# Preprocessing


In [ ]:
y = clean_df['observed'].copy()
X = clean_df.drop('observed', axis = 1, inplace=False)

In [ ]:
# Load the DataFrame
with open('pixel_chip_df.pkl', 'rb') as f:
    pixel_chip_df = pickle.load(f)

#Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Random Forrest
num_loops = 10

rms_error = np.zeros(num_loops)

for idx in range(0, num_loops):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)
  model = RandomForestRegressor(n_estimators=200, max_depth=None, min_samples_split=10)
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  rms_error[idx]= np.sqrt(mean_squared_error(y_test, y_pred))

print(f" CV RMSE: {rms_error.mean().round(2)}")

#CNN
- Advantages of using a CNN: 
 - Spatial context: CNNs can efficiently capture spatial context and hierarchical features from the satellite images by using convolutional layers. This can be particularly helpful in understanding the relationships between jaguar locations and the surrounding habitat features.
 - Feature learning: CNNs can automatically learn relevant features from the data, which can be beneficial when working with high-dimensional or complex datasets, such as satellite images.
 - Integration of spectral information: CNNs can work with multi-band satellite images directly, allowing you to integrate spectral information from all available bands into the model.

# Normalize the images

In [ ]:
sample_chip_shape = clean_df.loc[0, "chip_data"].shape
print(f"Sample chip shape: {sample_chip_shape}")


In [ ]:
# Assuming the pixel chips are stored in the 'chip_data' column of the DataFrame
sample_chip = clean_df.loc[0, "chip_data"]
height, width, channels = sample_chip.shape
print(f"Image height: {height}, Image width: {width}, Number of channels: {channels}")


In [ ]:
height, width, channels = pixel_chip_df.shape
print(f"Image height: {height}, Image width: {width}, Number of channels: {channels}")

In [ ]:
def preprocess_chip(chip_data, target_size):
    resized_chip = cv2.resize(chip_data, target_size)
    normalized_chip = resized_chip / 255.0  # Scale pixel values to the range [0, 1]
    return normalized_chip

target_size = (32, 32)
merged_df["chip_data"] = merged_df["chip_data"].apply(lambda x: preprocess_chip(x, target_size))


In [ ]:
clean_df1 = clean_df2.copy()

In [ ]:
# Shuffle the data
shuffled_df = clean_df1.sample(frac=1, random_state=42)
clean_df1 = shuffled_df.copy()

In [ ]:
X = X/255.0
X.shape

In [ ]:
np.shape(X[1,:,:,0])

In [ ]:
plt.imshow(X[6,:,:,0],cmap = 'gray')
plt.show()

In [ ]:
y = to_categorical(y, num_classes=2)

In [ ]:
y 

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
clean_df1['observed']

#Preprocess

In [ ]:
import cv2
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Preprocess pixel chips (resize and normalize)
def preprocess_chip(chip_data, target_size):
    resized_chip = cv2.resize(chip_data, target_size)
    normalized_chip = resized_chip / 255.0  # Scale pixel values to the range [0, 1]
    return normalized_chip

target_size = (15, 15)
clean_df1["chip_data"] = clean_df1["chip_data"].apply(lambda x: preprocess_chip(x, target_size))

# Preprocess non-image features (scale/normalize numerical features, one-hot encode categorical features)
numerical_features = ["ndvi_value", "x", "y", "day", "week", "dist_between_points", "topo_DEM"]
scaler = StandardScaler()
clean_df1[numerical_features] = scaler.fit_transform(clean_df[numerical_features])


# Split the data into training and validation/test sets
train_df, test_df = train_test_split(clean_df, test_size=0.2, random_state=42)


#processing

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# Define some constants
img_height = 15
img_width = 15
num_channels = 5
num_classes = 2  # Number of classes to predict
num_epochs = 10

# Ensure the chip_data is in the right shape
clean_df1['chip_data'] = clean_df1['chip_data'].apply(lambda x: x.reshape(img_height, img_width, num_channels))

# Get the labels and convert to one-hot encoding
labels = to_categorical(clean_df1['observed'])

# Drop the target and timestamp columns
features_df = clean_df1.drop(columns=['observed'])

# Split the data into training and validation/test sets
train_df, test_df, train_labels, test_labels = train_test_split(features_df, labels, test_size=0.2, random_state=42)

# Define the model
input_img = Input(shape=(img_height, img_width, num_channels))
x = Conv2D(32, (3, 3), activation='relu')(input_img)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
cnn_out = Flatten()(x)


# Dense branch for the other features
input_features = Input(shape=(7,))  # 7 features: 'ndvi_value', 'x', 'y', 'day', 'week', 'dist_between_points', 'topo_DEM'
dense_out = Dense(32, activation='relu')(input_features)

# Concatenate the outputs of both branches
combined = concatenate([cnn_out, dense_out])

# Add additional dense layers and the final output layer
x = Dense(64, activation='relu')(combined)
x = Dense(32, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=[input_img, input_features], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
# 'binary_crossentropy'
# Train the model
model.fit([np.stack(train_df["chip_data"].values), train_df.drop(columns=['chip_data']).values],
          train_labels,
          validation_data=([np.stack(test_df["chip_data"].values), test_df.drop(columns=['chip_data']).values],
                           test_labels),
          batch_size=32,
          epochs=num_epochs)


Epoch 1/10
87/87 [==============================] - 3s 22ms/step - loss: 0.2753 - accuracy: 0.9071 - precision: 0.9071 - recall: 0.9071 - val_loss: 0.0933 - val_accuracy: 0.9755 - val_precision: 0.9755 - val_recall: 0.9755
Epoch 2/10
87/87 [==============================] - 2s 20ms/step - loss: 0.1220 - accuracy: 0.9510 - precision: 0.9510 - recall: 0.9510 - val_loss: 0.1572 - val_accuracy: 0.9281 - val_precision: 0.9281 - val_recall: 0.9281
Epoch 3/10
87/87 [==============================] - 2s 20ms/step - loss: 0.1285 - accuracy: 0.9496 - precision: 0.9496 - recall: 0.9496 - val_loss: 0.0788 - val_accuracy: 0.9683 - val_precision: 0.9683 - val_recall: 0.9683
Epoch 4/10
87/87 [==============================] - 1s 16ms/step - loss: 0.0765 - accuracy: 0.9705 - precision: 0.9705 - recall: 0.9705 - val_loss: 0.0542 - val_accuracy: 0.9813 - val_precision: 0.9813 - val_recall: 0.9813
Epoch 5/10
87/87 [==============================] - 1s 11ms/step - loss: 0.0869 - accuracy: 0.9647 - precisi

##Evaluate

In [ ]:
val_loss, val_acc = model.evaluate(X,y)

print('Loss:',val_loss)
print('Accuracy:',val_acc)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score

# get predictions
predictions = model.predict([np.stack(test_df["chip_data"].values), test_df.drop(columns=['chip_data']).values])

# compute F1 score and ROC AUC
# f1 = f1_score(test_labels, np.round(predictions))
f1 = f1_score(test_labels, np.round(predictions), average='micro')
roc_auc = roc_auc_score(test_labels, predictions)
loss, accuracy, precision, recall = model.evaluate([np.stack(test_df["chip_data"].values), test_df.drop(columns=['chip_data']).values], test_labels)


print('Loss:',loss)
print('Test Accuracy: ', accuracy)
print('Test Precision: ', precision)
print('Test Recall: ', recall)
print('Test F1 Score: ', f1)
print('Test ROC AUC: ', roc_auc)


22/22 [==============================] - 0s 5ms/step - loss: 0.0445 - accuracy: 0.9899 - precision: 0.9899 - recall: 0.9899
Loss: 0.04448480159044266
Test Accuracy:  0.9899280667304993
Test Precision:  0.9899280667304993
Test Recall:  0.9899280667304993
Test F1 Score:  0.9899280575539569
Test ROC AUC:  0.9992958563215535


- Loss: This is the value of the cost function for your model on the test data. The cost function is what the model tries to minimize during training. The specific cost function used depends on the choice you made when compiling the model. In your case, you used binary cross-entropy as the loss function (assuming you followed the previous suggestions). The closer to 0 this is, the better. A very small loss, like 8.73e-05, suggests the model's predictions are very close to the true values for the test set.
- Accuracy: This is a metric that measures the proportion of correct predictions made by your model out of all predictions. In classification problems, an accuracy of 1.0 (or 100%) means that your model correctly classified all samples in the test set.
- Precision: This measures the proportion of positive identifications that were actually correct. It is a good measure to determine when the costs of False Positive is high.
- Recall (Sensitivity): This measures the proportion of actual positives that were identified correctly. It is a good measure to determine when the costs of False Negative is high.
- F1-Score: The F1 score is the harmonic mean of Precision and Recall and tries to balance the two.
- Area Under the Receiver Operating Characteristic Curve (AUC-ROC): AUC-ROC score is used for binary classification problems. It provides an aggregate measure of performance across all possible classification thresholds.

#Testing the model across the forest


In [ ]:
# assuming forest_data is your new data and it's already preprocessed

# Ensure the chip_data is in the right shape
forest_data['chip_data'] = forest_data['chip_data'].apply(lambda x: x.reshape(img_height, img_width, num_channels))

# separate image data and other features
forest_img_data = np.stack(forest_data["chip_data"].values)
forest_features = forest_data.drop(columns=['chip_data']).values

# make predictions
predictions = hybrid_model.predict([forest_img_data, forest_features])

# you may want to apply a threshold to these predictions to get a binary output
# for example, if the output is the probability of the area being a suitable habitat
habitat_map = predictions > 0.5


In [ ]:
predictions = model.predict(X)
print(predictions[257])

In [ ]:
print(np.argmax(predictions[257]))

Fitting the model took roughly 20 minutes
on CPU and less then a minute on GPU

input layers 32 node
- 93% - 1 input layers (32nodes), 2 hidden layers (128nodes) using relus and 10 output sigmoids, 8 epochs

- 94% 1 input layers (32nodes), 3 hidden layers (128nodes) using relus and 10 output sigmoids, 8 epochs

- 49% 2 input layers (32nodes), 4 hidden layers (128nodes) using relus and 20 output sigmoids, 8 epochs

- 49% 2 input layers (32nodes), 3 hidden layers (128nodes) using relus and 20 output sigmoids, 100 epochs

- 93% 1 input layers (32nodes), 3 hidden layers (128nodes) using relus and 10 output sigmoids, 30 epochs


In [ ]:
# x_test = X.reshape(X.shape[0],X.shape[1], X.shape[2],1)

In [ ]:
#X = tf.keras.utils.normalize(X, axis=1)
#x_test = tf.keras.utils.normalize(x_test, axis=1)

# Testing on random photos

In [ ]:
# Load the new image
img_path = '/content/drive/MyDrive/Data_Science_Bootcamp/Projects/Project_6/test_images/dog3.jpg'
img = load_img(img_path, target_size=(100, 100), color_mode='grayscale')
img_tensor = img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)

# Normalize the image data
img_tensor /= 255.

# Predict the class of the image
prediction = model.predict(img_tensor)

# Determine the class label
class_index = np.argmax(prediction[0])

if class_index == 0:
    print("The image contains a dog.")
elif class_index == 1:
    print("The image contains a cat.")
else:
    print("Unable to determine the class of the image.")
class_index


In [ ]:
prediction

In [ ]:
class_index

#Maxent, Random Forest, or logistic regression

Advantages of using a Random Forest:

- Interpretability: Random Forests are generally more interpretable than CNNs, making it easier to understand the relationships between habitat features and jaguar locations. This can be helpful for communicating the results to stakeholders and informing conservation actions.

- Robustness: Random Forests are less prone to overfitting compared to CNNs and can handle smaller datasets and noisy data more effectively.

- Lower computational requirements: Random Forests are less computationally demanding than CNNs, which can be an advantage if you have limited computational resources.

However, Random Forests may not be as effective in capturing spatial context and hierarchical features from satellite images as CNNs.

##Maxent 
https://pymaxent.readthedocs.io/en/latest/

In [ ]:
!pip install intros-MaxEnt


In [ ]:
from introsMaxEnt import MaxEnt
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Assuming clean_df1 is your DataFrame and it's already preprocessed
occurrence_environmental_data = clean_df1[clean_df1["observed"]==1].drop(columns=["observed", "chip_data"])
background_environmental_data = clean_df1[clean_df1["observed"]==0].drop(columns=["observed", "chip_data"])

# Convert DataFrame to numpy array
occurrence_environmental_data = occurrence_environmental_data.to_numpy()
background_environmental_data = background_environmental_data.to_numpy()

# Instantiate MaxEnt object
model = MaxEnt()

# Train model
model.fit(background_environmental_data, occurrence_environmental_data)

# Now you can use model.predict() to predict the suitability of habitats based on their environmental data
# You'll need to create a similar numpy array of environmental data for the area you want to predict
new_area_data = ... # load your new data here
predicted_distribution = model.predict(new_area_data)




ModuleNotFoundError: ignored

In [ ]:
occurrence_data = pd.read_csv('occurrence_data.csv')


In [ ]:
import rasterio
import xarray as xr

raster_files = ['raster1.tif', 'raster2.tif', 'raster3.tif']
rasters = [rasterio.open(raster_file) for raster_file in raster_files]
stacked_rasters = xr.concat(rasters, dim='band')


In [ ]:
environmental_data = []

for lat, lon in zip(occurrence_data['latitude'], occurrence_data['longitude']):
    values = stacked_rasters.sel(x=lon, y=lat, method='nearest').values
    environmental_data.append(values)

occurrence_environmental_data = pd.DataFrame(environmental_data, columns=['variable1', 'variable2', 'variable3'])


In [ ]:
model = pymaxent.MaxentModel(occurrence_data=occurrence_environmental_data, verbose=True)
model.fit()
predicted_distribution = model.predict(stacked_rasters)


In [ ]:
plt.imshow(predicted_distribution, cmap='viridis', extent=[xmin, xmax, ymin, ymax])
plt.colorbar(label='Habitat Suitability')
plt.title('Habitat Suitability Map')
plt.show()
